# Psyche-R1: Baseline vs LoRA Fine-Tuned (CPU-safe)

**Goal.** Show a simple, reproducible baseline vs fine-tuned comparison on a small mental-health counseling dataset (CounselChat), using a tiny instruct LLM (**Qwen2.5-0.5B-Instruct**) and **LoRA adapters** trained on **CPU**.

**What we’ll do**
1) Load & clean `counselchat-data.csv`
2) Build 80/10/10 train/dev/test splits
3) Baseline: zero-shot generation (no training)
4) LoRA fine-tune (CPU manual loop, small steps)
5) Evaluate: generate with the adapter, compare side-by-side
6) Export CSVs + quick empathy probe

> Note: CPU training is slow and minimal. This is for demonstration/assessment, not SOTA results.


In [1]:
import os, math, json, re, random, gc
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DATA_CSV_PATH = "../data/raw/counselchat-data.csv"
OUT_DIR = Path("runs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

c:\Users\User\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [3]:
def strip_html(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = re.sub(r"<br\s*/?>", "\n", s, flags=re.I)
    s = re.sub(r"<[^>]+>", "", s) 
    s = re.sub(r"\s+\n", "\n", s)
    return s.strip()

df = pd.read_csv(DATA_CSV_PATH)

candidate_cat_cols = [c for c in df.columns if c.lower() in {"questioncategory","category","tags","topics"}]
cat_col = candidate_cat_cols[0] if candidate_cat_cols else None

df["clean_answer"] = df["answerText"].astype(str).apply(strip_html)

def make_input(row):
    title = str(row.get("questionTitle","")).strip()
    body  = str(row.get("questionText","")).strip()
    if title and body and title not in body:
        return f"{title}\n\n{body}"
    return body or title

def make_instruction(row):
    base = "Provide an empathetic, evidence-based counseling reply."
    if cat_col and pd.notna(row.get(cat_col)) and str(row.get(cat_col)).strip():
        return f"{base} Topic: {str(row.get(cat_col)).strip()}."
    return base

usable = df[(df["clean_answer"].str.len() >= 20) & (df["questionText"].astype(str).str.len() >= 20)].copy()

records = []
for _, r in usable.iterrows():
    records.append({
        "instruction": make_instruction(r),
        "input": make_input(r),
        "output": r["clean_answer"]
    })

rng = np.random.RandomState(SEED)
idx = np.arange(len(records))
rng.shuffle(idx)
n = len(idx); n_train = int(0.8*n); n_dev = int(0.1*n)
train_idx = idx[:n_train]; dev_idx = idx[n_train:n_train+n_dev]; test_idx = idx[n_train+n_dev:]

splits = {
    "train": [records[i] for i in train_idx],
    "dev":   [records[i] for i in dev_idx],
    "test":  [records[i] for i in test_idx],
}

print({k: len(v) for k,v in splits.items()})

{'train': 1102, 'dev': 137, 'test': 139}


We use **Qwen/Qwen2.5-0.5B-Instruct** — a ~0.5B parameter instruct model suitable for CPU demos.

- MAX_LEN capped at 1024
- Batch sizes are small (CPU-friendly)
- We will do baseline generation with the base model (no training)


In [4]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  
USE_4BIT = True
USE_BF16 = True  
MAX_LEN = 1024
BATCH_SIZE = 2
GRAD_ACCUM = 16
EPOCHS = 1     
LR = 2e-4
WARMUP_RATIO = 0.03

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
#pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#pip install --upgrade bitsandbytes


SyntaxError: invalid syntax (214696030.py, line 1)

In [ ]:
USE_4BIT = True         
USE_BF16 = True          

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

def load_base_model(model_name: str, use_4bit: bool, use_bf16: bool):
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    bnb_cfg = None
    if use_4bit:
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )

    dtype_val = torch.bfloat16 if (use_bf16 and torch.cuda.is_available()) else None

    def _create_model(_bnb_cfg):
        return AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=_bnb_cfg,
            dtype=dtype_val
        )

    try:
        base_model = _create_model(bnb_cfg)
    except Exception as e:
        print("[WARN] 4-bit load failed, falling back to non-quantized FP16/BF16. Reason:")
        print("       ", repr(e))
        base_model = _create_model(None) 
        use_4bit = False

    base_model.eval()
    return tokenizer, base_model, use_4bit

tokenizer, base_model, USE_4BIT = load_base_model(MODEL_NAME, USE_4BIT, USE_BF16)
print("Loaded model. 4-bit active:", USE_4BIT, "| CUDA available:", torch.cuda.is_available())


c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5060 Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


[WARN] 4-bit load failed, falling back to non-quantized FP16/BF16. Reason:
        RuntimeError('CUDA error: no kernel image is available for execution on the device\nCUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\n')
Loaded model. 4-bit active: False | CUDA available: True


In [6]:
def format_prompt(inst: str, inp: str) -> str:
    if inp and inp.strip():
        return f"Instruction: {inst}\nInput: {inp}\nAnswer:"
    return f"Instruction: {inst}\nAnswer:"

@torch.no_grad()
def generate_responses(model, examples, max_new_tokens=256, do_sample=False, temperature=0.7, top_p=0.9):
    outputs = []
    for ex in examples:
        prompt = format_prompt(ex["instruction"], ex["input"])
        enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LEN).to(model.device)
        out = model.generate(**enc, max_new_tokens=max_new_tokens, do_sample=do_sample, temperature=temperature, top_p=top_p)
        text = tokenizer.decode(out[0], skip_special_tokens=True)
        pred = text[len(prompt):].strip()
        outputs.append(pred)
    return outputs

In [ ]:
base_model.to("cpu")

TEST_SAMPLES = 10 #10 rows
test_subset = splits["test"][:TEST_SAMPLES]

baseline_outputs = generate_responses(base_model, test_subset, do_sample=False)

baseline_df = pd.DataFrame({
    "instruction": [ex["instruction"] for ex in test_subset],
    "input":       [ex["input"] for ex in test_subset],
    "reference":   [ex["output"] for ex in test_subset],
    "baseline":    baseline_outputs
})

baseline_csv = OUT_DIR / "baseline_outputs.csv"
baseline_df.to_csv(baseline_csv, index=False)
print("Saved baseline outputs to:", baseline_csv)
baseline_df.head(3)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Saved baseline outputs to: runs\baseline_outputs.csv


,instruction,input,reference,baseline
0,"Provide an empathetic, evidence-based counseli...",Why do my boyfriend and I have such trouble co...,Try having a conversation with your boyfriend ...,Communication is key to any relationship. When...
1,"Provide an empathetic, evidence-based counseli...",My new husband constantly talks to himself\n\n...,Some people simply talk to themselves as a way...,It's important to recognize that communication...
2,"Provide an empathetic, evidence-based counseli...",How can I keep a long distance relationship go...,Hello. You are asking a very good question abo...,It is understandable that you are feeling anxi...


In [8]:
def build_hf_dataset(recs):
    return Dataset.from_list(recs)

def tokenize_sft(rec):
    inst, inp, out = rec["instruction"], rec["input"], rec["output"]
    prompt = format_prompt(inst, inp)
    full = prompt + " " + out + tokenizer.eos_token
    toks = tokenizer(full, max_length=MAX_LEN, truncation=True)
    toks["labels"] = toks["input_ids"].copy()
    return toks

train_ds = build_hf_dataset(splits["train"]).map(tokenize_sft, remove_columns=list(splits["train"][0].keys()))
dev_ds   = build_hf_dataset(splits["dev"]).map(tokenize_sft,   remove_columns=list(splits["dev"][0].keys()))
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

len(train_ds), len(dev_ds)

Map: 100%|██████████| 137/137 [00:00<00:00, 1313.65 examples/s]


(1102, 137)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

FORCE_CPU = True
USE_BF16 = False       
USE_4BIT = False        

def load_base_model_cpu_fp32(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cpu",
        torch_dtype=torch.float32   
    )

    model.eval()
    return tokenizer, model

tokenizer, train_model = load_base_model_cpu_fp32(MODEL_NAME)

def guess_lora_targets(m):
    names = [n for n, _ in m.named_modules()]
    keys = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "up_proj", "down_proj", "gate_proj",
        "c_attn", "c_proj",
        "w1", "w2", "w3"
    ]
    return sorted({k for k in keys if any(k in n for n in names)})

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=guess_lora_targets(train_model),
)

train_model = get_peft_model(train_model, lora_cfg)

train_model.train()
train_model.to("cpu")

trainable_params = [(n, p) for n, p in train_model.named_parameters() if p.requires_grad]
frozen_params = [(n, p) for n, p in train_model.named_parameters() if not p.requires_grad]

print(f"Total params: {len(trainable_params) + len(frozen_params)}")
print(f"Trainable params: {len(trainable_params)}")

if not trainable_params:
    raise RuntimeError("❌ No trainable parameters — LoRA didn't attach.")

print("First few trainable params:")
for n, _ in trainable_params[:10]:
    print("  ", n)

train_model.print_trainable_parameters()
print("LoRA attached successfully. Device:", next(train_model.parameters()).device)


Total params: 626
Trainable params: 336
✅ First few trainable params:
   base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
   base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
   base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497
LoRA attached successfully. Device: cpu


In [ ]:
import os
import math
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
os.environ["DISABLE_WANDB"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "" 

device = torch.device("cpu")
train_model.to(device)
train_model.train()

EPOCHS = 1
BATCH_SIZE = 1
GRAD_ACCUM = 16
LR = 2e-4
MAX_STEPS = None 

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collator
)

optim_params = [p for p in train_model.parameters() if p.requires_grad]
optimizer = AdamW(optim_params, lr=LR)

global_step = 0
optimizer.zero_grad()

print("Starting manual fine-tune on CPU:")
print(f"Trainable tensors: {len(optim_params)}")
print(f"Device: {device}")
print(f"Epochs: {EPOCHS}, Grad Accum: {GRAD_ACCUM}")

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1}/{EPOCHS} =====")
    for batch_idx, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = train_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
        )
        loss = outputs.loss

        (loss / GRAD_ACCUM).backward()

        if (batch_idx + 1) % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(optim_params, max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            print(f"[step {global_step}] loss={loss.item():.4f}")

            if MAX_STEPS is not None and global_step >= MAX_STEPS:
                break

    if MAX_STEPS is not None and global_step >= MAX_STEPS:
        break

if (batch_idx + 1) % GRAD_ACCUM != 0:
    torch.nn.utils.clip_grad_norm_(optim_params, max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()
    global_step += 1
    print(f"[final step {global_step}] (last partial batch)")

print("\nFinished CPU fine-tuning loop.")

adapter_out = OUT_DIR / "psyche_r1_sft"
adapter_out.mkdir(parents=True, exist_ok=True)
train_model.save_pretrained(str(adapter_out))
tokenizer.save_pretrained(str(adapter_out))
print(f"Saved fine-tuned LoRA adapter to: {adapter_out}")


🚀 Starting manual fine-tune on CPU...
Trainable tensors: 336
Device: cpu
Epochs: 1, Grad Accum: 16

===== Epoch 1/1 =====
[step 1] loss=2.9945
[step 2] loss=2.9017
[step 3] loss=2.7857
[step 4] loss=3.2014
[step 5] loss=2.5689
[step 6] loss=2.9188
[step 7] loss=2.3194
[step 8] loss=2.6603
[step 9] loss=2.7417
[step 10] loss=2.4911
[step 11] loss=2.4491
[step 12] loss=2.9548
[step 13] loss=2.6540
[step 14] loss=2.5123
[step 15] loss=2.7482
[step 16] loss=2.3909
[step 17] loss=2.1442
[step 18] loss=2.3488
[step 19] loss=2.3313
[step 20] loss=3.1156
[step 21] loss=2.6760
[step 22] loss=2.3375
[step 23] loss=2.5947
[step 24] loss=2.4627
[step 25] loss=2.5901
[step 26] loss=2.4313
[step 27] loss=2.6478
[step 28] loss=2.5955
[step 29] loss=1.7774
[step 30] loss=2.2323
[step 31] loss=2.3256
[step 32] loss=2.4782
[step 33] loss=2.3496
[step 34] loss=1.7259
[step 35] loss=2.5157
[step 36] loss=2.2343
[step 37] loss=2.9321
[step 38] loss=2.1441
[step 39] loss=2.1475
[step 40] loss=2.2275
[step 4

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM
from peft import PeftModel

ft_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cpu",          
    torch_dtype=torch.float32  
)

ft_model = PeftModel.from_pretrained(ft_model, str(OUT_DIR / "psyche_r1_sft"))
ft_model.eval()
ft_model.to("cpu")

print("Fine-tuned LoRA adapter loaded and ready for evaluation")

ft_outputs = generate_responses(
    ft_model,
    test_subset,
    do_sample=False  
)

compare_df = pd.DataFrame({
    "instruction": [ex["instruction"] for ex in test_subset],
    "input":       [ex["input"] for ex in test_subset],
    "reference":   [ex["output"] for ex in test_subset],
    "baseline":    baseline_df["baseline"],
    "finetuned":   ft_outputs
})

cmp_csv = OUT_DIR / "baseline_vs_finetuned.csv"
compare_df.to_csv(cmp_csv, index=False)

print("💾 Saved comparison to:", cmp_csv)
display(compare_df.head(3))


✅ Fine-tuned LoRA adapter loaded and ready for evaluation
💾 Saved comparison to: runs\baseline_vs_finetuned.csv


,instruction,input,reference,baseline,finetuned
0,"Provide an empathetic, evidence-based counseli...",Why do my boyfriend and I have such trouble co...,Try having a conversation with your boyfriend ...,Communication is key to any relationship. When...,"Hi, &nbsp;I'm so glad you're able to talk with..."
1,"Provide an empathetic, evidence-based counseli...",My new husband constantly talks to himself\n\n...,Some people simply talk to themselves as a way...,It's important to recognize that communication...,It sounds like you are experiencing some anxie...
2,"Provide an empathetic, evidence-based counseli...",How can I keep a long distance relationship go...,Hello. You are asking a very good question abo...,It is understandable that you are feeling anxi...,It sounds like you are in a difficult situatio...
